In [1]:
include("io_equation.jl")

$\require{action}$

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) GAP, Hecke, Nemo, Polymake and Singular
Version 0.3.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2020 by The Oscar Development Team


find_ioequation (generic function with 2 methods)

**Example 1:** Just take predator-prey and make both variables to be outputs. Then it would be natural that the io-equations are the original equations after renaming x's to y's. And this is what happens.

In [2]:
logger = Logging.SimpleLogger(stdout, Logging.Debug)
global_logger(logger)

varnames = ["x1", "x2", "a", "b", "c", "d"]
R, (x1, x2, a, b, c, d) = PolynomialRing(QQ, varnames)

ode = ODE(
    Dict(
        x1 => a * x1 - b * x1 * x2,
        x2 => -c * x2 + d * x1 * x2
    ), []
)

@time io_equations = find_ioequation(ode, [x1, x2])

┌ Debug: 	 Computing Lie derivatives 2020-09-12T22:51:34.024
└ @ Main /home/gleb/code/EliminationODE/io_equation.jl:83
┌ Debug: Current degrees of io-equations [(1, [1]), (2, [1])]
└ @ Main /home/gleb/code/EliminationODE/io_equation.jl:123
┌ Debug: Orders: [0, 0]
└ @ Main /home/gleb/code/EliminationODE/io_equation.jl:124
┌ Debug: Sizes: [2, 2]
└ @ Main /home/gleb/code/EliminationODE/io_equation.jl:125
┌ Debug: Scores: [(1, -1, 2, 1), (1, -1, 2, 2)]
└ @ Main /home/gleb/code/EliminationODE/io_equation.jl:136
┌ Debug: Prolonging output number 1
└ @ Main /home/gleb/code/EliminationODE/io_equation.jl:139
┌ Debug: 	 Decreasing degree with linear combination 2020-09-12T22:51:36.442
└ @ Main /home/gleb/code/EliminationODE/elimination.jl:237
┌ Debug: 	 Using GCD to eliminate extra factors 2020-09-12T22:51:37.105
└ @ Main /home/gleb/code/EliminationODE/elimination.jl:317
┌ Debug: Generating new point on the variety
└ @ Main /home/gleb/code/EliminationODE/elimination.jl:170
┌ Debug: Generating ne

2-element Array{Any,1}:
 -a*y1_0 + b*y1_0*y2_0 + y1_1
 c*y2_0 - d*y1_0*y2_0 + y2_1

**Example 2:** This example (aritificial) demonstrates the insufficiency of the generated io-equations for identifiability purposes.

In [3]:
R, (x1, x2, b) = PolynomialRing(QQ, ["x1", "x2", "b"])

ode = ODE(Dict(
    x1 => (1 + x1^2) // 2,
    x2 => (1 - x1^2) // (1 + x1^2)
), [])

@time io_equations = find_ioequation(ode, [2 * x1 // (b * (1 + x1^2)), x2])

┌ Debug: 	 Computing Lie derivatives 2020-09-12T22:53:38.936
└ @ Main /home/gleb/code/EliminationODE/io_equation.jl:83
┌ Debug: Current degrees of io-equations [(1, [2]), (2, [1])]
└ @ Main /home/gleb/code/EliminationODE/io_equation.jl:123
┌ Debug: Orders: [0, 0]
└ @ Main /home/gleb/code/EliminationODE/io_equation.jl:124
┌ Debug: Sizes: [3, 2]
└ @ Main /home/gleb/code/EliminationODE/io_equation.jl:125
┌ Debug: Scores: [(2, -1, 3, 1), (1, -1, 2, 2)]
└ @ Main /home/gleb/code/EliminationODE/io_equation.jl:136
┌ Debug: Prolonging output number 2
└ @ Main /home/gleb/code/EliminationODE/io_equation.jl:139
┌ Debug: 	 Decreasing degree with linear combination 2020-09-12T22:53:38.996
└ @ Main /home/gleb/code/EliminationODE/elimination.jl:243
┌ Debug: 	 Using GCD to eliminate extra factors 2020-09-12T22:53:38.996
└ @ Main /home/gleb/code/EliminationODE/elimination.jl:317
┌ Debug: Generating new point on the variety
└ @ Main /home/gleb/code/EliminationODE/elimination.jl:170
┌ Debug: Generating ne

2-element Array{Any,1}:
 b^2*y1_0^2 + b^2*y1_1^2 - 1
 -4*b^2*y1_0^2 - 4*y2_1^2 + 4

In these equations, $b$ always appears squared, so one might conclude that $b$ is only locally identifiable.

*However*, direct computation shows that $b y_1' - y_2' = 0$ is a consequence of the system. Therefore, $b$ actually is identifiable. This is because the variety defined by these io-equations is not prime. Indeed:

In [4]:
4 * io_equations[1] + io_equations[2]

4*b^2*y1_1^2 - 4*y2_1^2

This factors! And one of the factors is the above consequence.

**Question:** Would the following help?
* Consider a new variable $z = y_1' + y_2'$ 
* Use the io-equations to eliminate $y_1'$ and $y_2'$ from the above equation thus getting an equation in $z, y_1, y_2$, it will likely factor, we leave only the true factor
* Considering its coefficients as well